In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../")

In [3]:
# !python -m pip install -r ../requirements.txt

In [4]:
from src.model import TrashSegmentation
from src.data import UAVVaste

from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger

In [5]:
from torchvision import transforms
import torch
import pytorch_lightning as pl
from pathlib import Path

In [6]:
val_size = 0.2
batch_size = 2
img_size = (512, 512)
CONFIG = {
    "img_size": img_size,
    "batch_size": batch_size,
    "val_size": val_size,
}

# DataLoader

In [7]:
transform = preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(img_size),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
train_set = UAVVaste(
    "../data/raw/UAVVaste/annotations/annotations.json",
    "../data/raw/UAVVaste/images-small",
    transform=transform, 
    to_rotate=True,
    train=True,
)
val_set = UAVVaste(
    "../data/raw/UAVVaste/annotations/annotations.json",
    "../data/raw/UAVVaste/images-small",
    transform=transform, 
    to_rotate=True,
    train=True,
)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [12]:
train_set[1]['mask'].shape

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [28]:
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)

# Model

In [14]:
model = TrashDetector()

In [15]:
logger = TensorBoardLogger("../logs", name="UAVVaste")
trainer = pl.Trainer(logger=logger)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [16]:
trainer.fit(model, train_dataloader, val_dataloader)


  | Name  | Type      | Params
------------------------------------
0 | model | DeepLabV3 | 11.0 M
------------------------------------
11.0 M    Trainable params
0         Non-trainable params
11.0 M    Total params
44.081    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

mIoU = 0.012976600788533688
mDSC = 0.025590332224965096


Training: -1it [00:00, ?it/s]

/home/renesat/0. PARA/1. Projects/Ecodrone/NN/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1051: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
